#importing basic packages

In [ ]:
import re
import sys
import time
import datetime
import requests

# Renew saved coockie for the request

In [ ]:
def get_cookie_value(r):
    return {'B': r.cookies.get('B')}

# Scrape the data from the HTML file

In [ ]:
def get_page_data(symbol):
    url = "https://finance.yahoo.com/quote/%s/?p=%s" % (symbol, symbol)
    r = requests.get(url)
    cookie = get_cookie_value(r)
    lines = r.content.decode('unicode-escape').strip(). replace('}', '\n')
    return cookie, lines.split('\n')

# Vliadte crumb not exist

In [ ]:
def find_crumb_store(lines):
    # Looking for
    # ,"CrumbStore":{"crumb":"9q.A4D1c.b9
    for l in lines:
        if re.findall(r'CrumbStore', l):
            return l
    print("Did not find CrumbStore")

# Parse crumb if exist

In [ ]:
def split_crumb_store(v):
    return v.split(':')[2].strip('"')

# Get the coockie for the api request

In [ ]:
def get_cookie_crumb(symbol):
    cookie, lines = get_page_data(symbol)
    crumb = split_crumb_store(find_crumb_store(lines))
    return cookie, crumb

# Pull out the historical data and write it to the CSV file

In [ ]:
def get_data(symbol, start_date, end_date, cookie, crumb):
    filename = '%s.csv' % (symbol)
    url = "https://query1.finance.yahoo.com/v7/finance/download/%s?period1=%s&period2=%s&interval=1d&events=history&crumb=%s" % (symbol, start_date, end_date, crumb)
    response = requests.get(url, cookies=cookie)
    with open (filename, 'wb') as handle:
        for block in response.iter_content(1024):
            handle.write(block)

# Get time periods by time format for ~10000 data rows

In [ ]:
def get_now_epoch():
    # @see https://www.linuxquestions.org/questions/programming-9/python-datetime-to-epoch-4175520007/#post5244109
    return int(time.time())

# Scraping the data with the help of the functions above

In [ ]:
def scrape_quotes(symbol):
    start_date = 0
    end_date = get_now_epoch()
    cookie, crumb = get_cookie_crumb(symbol)
    get_data(symbol, start_date, end_date, cookie, crumb)
